In [1]:
import os
import pandas as pd
import numpy as np
from keras.applications.resnet_v2 import ResNet50V2
import tensorflow as tf
from tensorflow.keras import models, Sequential, layers, optimizers, callbacks
from tensorflow.keras.layers import Dense, Reshape
from keras.preprocessing import image 
from tqdm import tqdm

2023-06-09 11:04:09.622196: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-09 11:04:09.694755: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-09 11:04:09.697092: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-09 11:04:11.209193: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
os.getcwd()

'/home/edloftus/code/AtinDhawan/Dog_Identifier'

In [8]:
group0 = os.listdir('/home/edloftus/code/AtinDhawan/Dog_Identifier/raw_data/Train Grouped/Herding')
group1 = os.listdir('/home/edloftus/code/AtinDhawan/Dog_Identifier/raw_data/Train Grouped/Hound')
group2 = os.listdir('/home/edloftus/code/AtinDhawan/Dog_Identifier/raw_data/Train Grouped/Non-Sporting')
group3 = os.listdir('/home/edloftus/code/AtinDhawan/Dog_Identifier/raw_data/Train Grouped/Sporting')
group4 = os.listdir('/home/edloftus/code/AtinDhawan/Dog_Identifier/raw_data/Train Grouped/Terrier')
group5 = os.listdir('/home/edloftus/code/AtinDhawan/Dog_Identifier/raw_data/Train Grouped/Toy')
group6 = os.listdir('/home/edloftus/code/AtinDhawan/Dog_Identifier/raw_data/Train Grouped/Working')

In [31]:
list_of_cats = []
for i_1 in [group0, group1, group2, group3, group4, group5, group6]:
    group = []
    for i_2 in i_1:
        if i_2.startswith('n'):
            group.append(i_2[:9])
    list_of_cats.append(group)
grp0 = list_of_cats[0]
grp1 = list_of_cats[1]
grp2 = list_of_cats[2]
grp3 = list_of_cats[3]
grp4 = list_of_cats[4]
grp5 = list_of_cats[5]
grp6 = list_of_cats[6]
all_grps = grp0 + grp1 + grp2 + grp3 + grp4 + grp5 + grp6

In [32]:
def subcat(x):
    if x[:9] in grp0: return 0
    elif x[:9] in grp1: return 1
    elif x[:9] in grp2: return 2
    elif x[:9] in grp3: return 3
    elif x[:9] in grp4: return 4    
    elif x[:9] in grp5: return 5    
    elif x[:9] in grp6: return 6

In [43]:
file_names_train = os.listdir('/home/edloftus/code/AtinDhawan/Dog_Identifier/raw_data/Attempt_2/All_Train')
file_names_test = os.listdir('/home/edloftus/code/AtinDhawan/Dog_Identifier/raw_data/Attempt_2/All_Test')

In [44]:
train_breed_cat = {}
for file in file_names_train:
    if file[0:9] in all_grps:
        train_breed_cat['/home/edloftus/code/AtinDhawan/Dog_Identifier/raw_data/Attempt_2/All_Train/' + file] = subcat(file)
print(len(train_breed_cat))
test_breed_cat = {}
for file in file_names_test:
    if file[0:9] in all_grps:
        test_breed_cat['/home/edloftus/code/AtinDhawan/Dog_Identifier/raw_data/Attempt_2/All_Test/' + file] = subcat(file)
len(test_breed_cat)

11500


8301

In [49]:
train_breed_cat

{'/home/edloftus/code/AtinDhawan/Dog_Identifier/raw_data/Attempt_2/All_Train/n02094114_487.jpg': 4,
 '/home/edloftus/code/AtinDhawan/Dog_Identifier/raw_data/Attempt_2/All_Train/n02091134_19308.jpg': 1,
 '/home/edloftus/code/AtinDhawan/Dog_Identifier/raw_data/Attempt_2/All_Train/n02094258_739.jpg': 4,
 '/home/edloftus/code/AtinDhawan/Dog_Identifier/raw_data/Attempt_2/All_Train/n02104365_9004.jpg': 2,
 '/home/edloftus/code/AtinDhawan/Dog_Identifier/raw_data/Attempt_2/All_Train/n02110185_1532.jpg': 6,
 '/home/edloftus/code/AtinDhawan/Dog_Identifier/raw_data/Attempt_2/All_Train/n02100583_2681.jpg': 3,
 '/home/edloftus/code/AtinDhawan/Dog_Identifier/raw_data/Attempt_2/All_Train/n02099849_2648.jpg': 3,
 '/home/edloftus/code/AtinDhawan/Dog_Identifier/raw_data/Attempt_2/All_Train/n02095889_3694.jpg': 4,
 '/home/edloftus/code/AtinDhawan/Dog_Identifier/raw_data/Attempt_2/All_Train/n02111889_1314.jpg': 6,
 '/home/edloftus/code/AtinDhawan/Dog_Identifier/raw_data/Attempt_2/All_Train/n02096585_32.jp

In [45]:
model = tf.keras.Sequential()

model.add(layers.Conv2D(16, kernel_size = (3,3), padding = 'same', activation = 'relu', input_shape = (224,224,3)))
# model.add(layers.MaxPool2D(pool_size=(2,2))) 

model.add(layers.Conv2D(32, kernel_size = (3,3), padding = 'same', activation = 'relu'))
model.add(layers.MaxPool2D(pool_size=(2,2))) 

model.add(layers.Conv2D(64, kernel_size = (3,3), padding = 'same', activation = 'relu'))
model.add(layers.MaxPool2D(pool_size=(2,2))) 

model.add(layers.Conv2D(128 ,kernel_size = (3,3) , padding = 'same' , activation='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2)))     
          
model.add(layers.Flatten())
model.add(layers.Dense(7, activation = 'softmax'))

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 224, 224, 16)      448       
                                                                 
 conv2d_13 (Conv2D)          (None, 224, 224, 32)      4640      
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 112, 112, 32)     0         
 g2D)                                                            
                                                                 
 conv2d_14 (Conv2D)          (None, 112, 112, 64)      18496     
                                                                 
 max_pooling2d_11 (MaxPoolin  (None, 56, 56, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_15 (Conv2D)          (None, 56, 56, 128)      

In [46]:
def load_image(folder_name):
    image = tf.io.read_file(folder_name)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [224,224])    
    return image

In [48]:
lst_of_imgs = []
for i in tqdm(train_breed_cat.keys()):
    lst_of_imgs.append(load_image(i))

100%|█████████████████████████████████████████████████████████████████████████████| 11500/11500 [02:58<00:00, 64.56it/s]


In [ ]:
lst_of_imgs_test = []
for i in tqdm(test_breed_cat.keys()):
    lst_of_imgs_test.append(load_image(i))

 13%|█████████▌                                                                  | 1046/8301 [04:37<37:37:49, 18.67s/it]